In [18]:
import loader
import pandas as pd

In [26]:
root = "datasets3/"
test_interactions = pd.read_csv(root+"hidden_interactions.csv")

recs_file = "svd_recommendations.json"
#recs_file = "baseline_recommendations.json"

recs = loader.load_user_likes(recs_file)

In [27]:
users_test_ratings = test_interactions.groupby('u').apply(
    lambda df: dict(zip(df['i'], df['rating']))
).to_dict()

/tmp/ipykernel_42036/2354451776.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  users_test_ratings = test_interactions.groupby('u').apply(


In [28]:
# ---- Evaluation ----
hit_scores = {}  # per-user hit rate
neg_count = 0

for user_id, recipe_ratings in users_test_ratings.items():
    recommended_items = recs[str(user_id)]  # convert to string if your JSON uses string keys
    
    score = 0.0

    # count how many test recipes appear in recommendations
    for recipe_id, rating in recipe_ratings.items():
        if recipe_id in recommended_items:
            # Scale hits by rating
            if rating == 5:
                score += 1
            elif rating == 4:
                score += 0.5
            elif rating == 3:
                score += 0
            elif rating == 2:
                neg_count -= 0.5
                score -= 0.5
            elif rating == 1:
                neg_count -= 1
                score -= 1

    # divide by number of test recipes to get per-user hit rate
    num_recipes = len(recipe_ratings)
    if num_recipes > 0:
        hit_scores[int(user_id)] = score / num_recipes


# ---- Aggregate metrics ----
overall_hit_rate = sum(hit_scores.values()) / len(hit_scores)
print("Overall Hit Rate:", overall_hit_rate)
print(neg_count)

Overall Hit Rate: 0.009968242766407905
0
